In [80]:
import yaml
from pprint import pprint
import dicttoxml
# there is also dict2xml available in pypy (python3)
import xml.etree.ElementTree as ET


In [81]:
with open('../data/zynqmon.yml') as f:
    y = yaml.load(f, Loader=yaml.FullLoader)
    pprint(y)


{'cm_hw_rev': '1A',
 'config': [{'count': 25,
             'extra': 'Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;',
             'mcu_call': 'firefly_temps',
             'name': 'firefly',
             'names': ['K01  12 Tx GTH',
                       'K01  12 Rx GTH',
                       'K02  12 Tx GTH',
                       'K02  12 Rx GTH',
                       'K03  12 Tx GTH',
                       'K03  12 Rx GTH',
                       'K04 4 XCVR GTY',
                       'K05 4 XCVR GTY',
                       'K06 4 XCVR GTY',
                       'K07  12 Tx GTY',
                       'K07  12 Rx GTY',
                       'V01 4 XCVR GTY',
                       'V02 4 XCVR GTY',
                       'V03 4 XCVR GTY',
                       'V04 4 XCVR GTY',
                       'V05 4 XCVR GTY',
                       'V06 4 XCVR GTY',
                       'V07 4 XCVR GTY',
                       'V08 4 XCVR GTY',
                       'V09 4

In [82]:
# This is the parent (root) tag
# onto which other tags would be
# created
data = ET.Element('node')
data.set('id', 'PL_MEM')


In [83]:
cm = ET.SubElement(data, 'node')
cm.set('id', 'CM')
cm.set('addr', '0x00000000')


In [84]:
config = y['config']

for c in config: # loop over entries in configuration (sensor category)
    print("here", c['name'])
    if not 'start' in c:
        pprint(c)
    start = c['start']
    count = c['count']
    i = 0 # counter over the number of sensors within a category
    names = c['names']
    if 'prefixes' in c:
        prefixes = c['prefixes']
    else:
        prefixes = ' '
    for p in prefixes:
        for n in names: # loop over sensor names
            node = ET.SubElement(cm, 'node')
            if ( len(p.lstrip()) > 0 ):
                id = p +"_" +  n.replace(' ', '_')
            else:
                id = n.replace(' ', '_')
            node.set('id', id)
            addr = int((start + i)/2)
            remain = (start + i) % 2
            node.set('addr', str(hex(addr)))
            i += 1 # offset for next sensor
            # this appears to be on all the nodes
            node.set("permission", "r")
            # set parameter based on the type
            if ( c['type'] == 'int8' ):
                width=8
                format = "d"
            elif ( c['type'] == 'fp16' ):
                width=16
                format = "fp16"
            elif ( c['type'] == 'char' ):
                width=8
                format = "c"
            elif ( c['type'] == 'uint32_t' ):
                width=32
                format = "u"
            else:
                print("ERROR: unknown type", c['type'])
                break
            mask = (1 << width) - 1
            if (remain == 0):
                node.set('mask', "0x{0:08X}".format(mask))
            else:
                node.set('mask', "0x{0:08X}".format(mask<<16))
            if 'extra' in c:
                extra = c['extra']
            else:
                extra = ""
            node.set('parameters', "format=" + format + ";" + extra)
        if 'size' in c:
            print("size:", c['size'], c['name'])
            node.set('size', str(hex(c['size'])))


here firefly
here psmon
here adcmon
here uptime
here gitversion
size: 5 gitversion
here fpga


In [85]:
s_xml = ET.tostring(data)


In [86]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(s_xml, 'xml')
pretty_xml = bs.prettify()
print(pretty_xml)


<?xml version="1.0" encoding="utf-8"?>
<node id="PL_MEM">
 <node addr="0x00000000" id="CM">
  <node addr="0x0" id="K01__12_Tx_GTH" mask="0x000000FF" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node addr="0x0" id="K01__12_Rx_GTH" mask="0x00FF0000" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node addr="0x1" id="K02__12_Tx_GTH" mask="0x000000FF" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node addr="0x1" id="K02__12_Rx_GTH" mask="0x00FF0000" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node addr="0x2" id="K03__12_Tx_GTH" mask="0x000000FF" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node addr="0x2" id="K03__12_Rx_GTH" mask="0x00FF0000" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node addr="0x3" id="K04_4_XCVR_GTY" mask="0x000